<a href="https://colab.research.google.com/github/Ashish-Soni08/Playground/blob/main/haystack/Advent_of_Haystack_Prompt_Engineering_Challenge(Ashish_Soni).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advent of Haystack - Day 3
_Make a copy of this Colab to start!_

Here, you'll be provided a nearly complete RAG pipeline that is supposed to do QA on a number of URLs. Our aim is to create a [`PromptBuilder`](https://docs.haystack.deepset.ai/v2.0/docs/promptbuilder) that uses a template which can produce answers with references as to where the answer is coming from.

1. **Run the indexing pipeline:** This is already complete. Here, we are writing the contents of various haystack documentation pages into an `InMemoryDocumentStore`. We are also creating embeddings for our documents with a `SentenceTransformersDocumentEmbedder`
2. **Your task is to complete step 2 👇**

#Installation
**Note:** There is a known issue with colab due to a version conflict error related to `llmx` which comes with Colab. You might get an `llmx` error. You can safely ignore this, or run `pip uninstall -y llmx`

In [1]:
%%capture

!pip install haystack-ai
!pip install boilerpy3
!pip install transformers accelerate bitsandbytes sentence_transformers

## 1) Write Documents to InMemoryDocumentStore

Here, we are writing the contents of a few URLs into an `InMemoryDocumentStore`

In [2]:
from haystack import Pipeline
from haystack.document_stores import InMemoryDocumentStore
from haystack.components.fetchers import LinkContentFetcher
from haystack.components.converters import HTMLToDocument
from haystack.components.preprocessors import DocumentSplitter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter


document_store = InMemoryDocumentStore()

link_fetcher = LinkContentFetcher()
converter = HTMLToDocument()
splitter = DocumentSplitter(split_length=100, split_overlap=5)
embedder = SentenceTransformersDocumentEmbedder()
writer = DocumentWriter(document_store=document_store)

indexing_pipeline = Pipeline()
indexing_pipeline.add_component("link_fetcher", link_fetcher)
indexing_pipeline.add_component("converter", converter)
indexing_pipeline.add_component("splitter", splitter)
indexing_pipeline.add_component("embedder", embedder)
indexing_pipeline.add_component("writer", writer)

indexing_pipeline.connect("link_fetcher", "converter")
indexing_pipeline.connect("converter", "splitter")
indexing_pipeline.connect("splitter", "embedder")
indexing_pipeline.connect("embedder", "writer")

In [3]:
indexing_pipeline.run(data={"link_fetcher":{"urls": ["https://docs.haystack.deepset.ai/v2.0/docs/sentencetransformerstextembedder", "https://docs.haystack.deepset.ai/v2.0/docs/openaidocumentembedder"]}})

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'writer': {'documents_written': 7}}

## 2) Build a RAG Pipeline
Here, we have provided a nearly complete RAG pipeline, but the `PromptBuilder` is mising. Create one and add it to the pipeline. Make sure your `PromptBuilder` is able to use the `url` from the documents metadata. That way, you can ask for a response that includes references!


In [4]:
from getpass import getpass

api_key = getpass("Enter OpenAI Api key: ")

Enter OpenAI Api key: ··········


In [18]:
document_store.storage

{'0da0842b0156b3c3f55b4f1144fcec9413be1838cceaa3708843e9eb9f26951a': Document(id=0da0842b0156b3c3f55b4f1144fcec9413be1838cceaa3708843e9eb9f26951a, content: 'Enabling GPU Acceleration
 SentenceTransformersTextEmbedder
 SentenceTransformersTextEmbedder transfor...', meta: {'content_type': 'text/html', 'url': 'https://docs.haystack.deepset.ai/v2.0/docs/sentencetransformerstextembedder', 'source_id': '5f6b19c3f8b8faccc7033b2fb49ea55f7dba4214ad63281c99defac513947f8b'}, embedding: vector of size 768),
 '667bbf1de076e41d2b30dd44ccedf0f0512d8effce907b9ebb57e616430e9956': Document(id=667bbf1de076e41d2b30dd44ccedf0f0512d8effce907b9ebb57e616430e9956, content: 'the computed embedding, known as vector.
 Compatible Models
 Unless specified otherwise while initiali...', meta: {'content_type': 'text/html', 'url': 'https://docs.haystack.deepset.ai/v2.0/docs/sentencetransformerstextembedder', 'source_id': '5f6b19c3f8b8faccc7033b2fb49ea55f7dba4214ad63281c99defac513947f8b'}, embedding: vector of size 768)

In [27]:
import torch

from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers import InMemoryEmbeddingRetriever
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.generators import GPTGenerator

######## Complete this section #############
prompt_template = """ According to these documents,
answer the given question in a structured and comprehensive way.

{% for doc in documents %}
  {{ doc.content }} URL:{{ doc.meta['url'] }}
{% endfor %}


If the answer is contained in the documents, also report the source URL.
If the answer cannot be deduced from the documents, do not give an answer.

Question: {{question}}
Answer:
"""
prompt_builder = PromptBuilder(prompt_template)
############################################

query_embedder = SentenceTransformersTextEmbedder()
retriever = InMemoryEmbeddingRetriever(document_store=document_store, top_k=2)
gpt_llm = GPTGenerator(api_key=api_key)

In [28]:
# Creating the Pipeline
pipeline = Pipeline()

pipeline.add_component(name="prompt_builder", instance=prompt_builder)
pipeline.add_component(name="query_embedder", instance=query_embedder)
pipeline.add_component(name="retriever", instance=retriever)
pipeline.add_component(name="llm", instance=gpt_llm)

In [29]:
# Connect the components in the Pipeline

pipeline.connect("query_embedder.embedding", "retriever.query_embedding")
pipeline.connect("retriever.documents", "prompt_builder.documents")
pipeline.connect("prompt_builder", "llm")

In [23]:
question = "How do I enable GPU acceleration?"

result = pipeline.run(data={"query_embedder": {"text": question}, "prompt_builder": {"question": question}})

print(result['llm']['replies'][0])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

To enable GPU acceleration, you can use the OpenAIDocumentEmbedder and SentenceTransformersTextEmbedder components in the Haystack framework.

The OpenAIDocumentEmbedder component computes the embeddings of a list of Documents and stores the obtained vectors in the embedding field of each Document. It uses OpenAI embedding models. You can use this component by including it before a DocumentWriter in an indexing pipeline. The output of this component includes the list of Documents enriched with embeddings. To enable GPU acceleration, you can follow the instructions provided by the OpenAI documentation (source URL: https://docs.haystack.deepset.ai/v2.0/docs/openaidocumentembedder).

The SentenceTransformersTextEmbedder component transforms a string into a vector that captures its semantics using an embedding model compatible with the Sentence Transformers library. You can use this component by including it before an embedding Retriever in a Query/RAG pipeline. The output of this componen

In [30]:
question = "How do I use the openai embedder?"

result = pipeline.run(data={"query_embedder": {"text": question}, "prompt_builder": {"question": question}})

print(result['llm']['replies'][0])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

To use the OpenAIDocumentEmbedder, you can follow these steps:

1. Import the necessary classes and packages:
   ```python
   from haystack import Document
   from haystack.components.embedders import OpenAIDocumentEmbedder
   ```

2. Create a Document object, providing the text and any relevant metadata:
   ```python
   doc = Document(text="some text",
                  metadata={"title": "relevant title",
                            "page number": 18})
   ```

3. Initialize an instance of the OpenAIDocumentEmbedder:
   ```python
   embedder = OpenAIDocumentEmbedder(metadata_fields_to_embed=["title"])
   ```
   The `metadata_fields_to_embed` parameter specifies the metadata fields that you want to include in the embedding.

4. Run the embedding process by passing a list of documents to the `run()` method:
   ```python
   docs_w_embeddings = embedder.run(documents=[doc])["documents"]
   ```
   This will embed the provided documents and store the computed vectors in the `embedding` fiel

Haystack is model-agnostic, which also means you can easily switch between different model providers. For example, instead of using an OpenAI model via an API, you can also try using an open source model running in this colab notebook. You can replace the `llm` with the one below. This might take up more resources in Colab. You might notice that models don't perform the same way, which can mean you need to change your prompt. It's ok to change the task from doing referenced QA to someting else. For example, we're also happy with a poem about the Haystack docs 🤗
```python
from haystack.components.generators import HuggingFaceLocalGenerator
llm = HuggingFaceLocalGenerator("HuggingFaceH4/zephyr-7b-beta",
                                 huggingface_pipeline_kwargs={"device_map":"auto",
                                               "model_kwargs":{"load_in_4bit":True,
                                                "bnb_4bit_use_double_quant":True,
                                                "bnb_4bit_quant_type":"nf4",
                                                "bnb_4bit_compute_dtype":torch.bfloat16}},
                                 generation_kwargs={"max_new_tokens": 350})
llm.warm_up()
```

In [24]:
from haystack.components.generators import HuggingFaceLocalGenerator
llm = HuggingFaceLocalGenerator("HuggingFaceH4/zephyr-7b-beta",
                                 huggingface_pipeline_kwargs={"device_map":"auto",
                                               "model_kwargs":{"load_in_4bit":True,
                                                "bnb_4bit_use_double_quant":True,
                                                "bnb_4bit_quant_type":"nf4",
                                                "bnb_4bit_compute_dtype":torch.bfloat16}},
                                 generation_kwargs={"max_new_tokens": 350})
llm.warm_up()

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [25]:
pipeline = Pipeline()
pipeline.add_component(instance=query_embedder, name="query_embedder")
pipeline.add_component(instance=retriever, name="retriever")
pipeline.add_component(instance=prompt_builder, name="prompt_builder")
pipeline.add_component(instance=llm, name="llm")

pipeline.connect("query_embedder.embedding", "retriever.query_embedding")
pipeline.connect("retriever.documents", "prompt_builder.documents")
pipeline.connect("prompt_builder", "llm")


In [26]:
query = "How do I use the openai embedder?"
result = pipeline.run(data={"query_embedder": {"text": query}, "prompt_builder": {"question": query}})
print(result['llm']['replies'][0])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


 To use the OpenAI embedder in Haystack, you can follow these steps:

1. Install the required packages:

   ```bash
   pip install openai haystack
   ```

2. Create a new Haystack pipeline and add the OpenAI embedder component:

   ```python
   from haystack import Document, Pipeline
   from haystack.components.embedders import OpenAIDocumentEmbedder

   pipeline = Pipeline(
       components=[
           Document(content="I love pizza!"),
           OpenAIDocumentEmbedder(api_key="YOUR-API-KEY"),
       ]
   )

   results = pipeline.run()
   ```

3. In the above example, replace `YOUR-API-KEY` with your actual OpenAI API key.

4. You can also embed metadata along with the text of the document by passing the `metadata_fields_to_embed` parameter to the OpenAI embedder component:

   ```python
   from haystack import Document, Pipeline
   from haystack.components.embedders import OpenAIDocumentEmbedder

   document = Document(
       content="I love pizza!",
       metadata={"title": "My